In [ ]:
import pandas as pd

train = pd.read_csv(r'datafile/titanic_train.csv')
test = pd.read_csv(r'datafile/titanic_test.csv')

def feature_engineering(df):
    # Sex
    df['Sex'] = df['Sex'].map({'female': 0, 'male': 1})

    # Embarked
    df.Embarked.fillna('S', inplace=True)
    df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

    # Title
    df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.')
    df['Title'] = df['Title'].replace(['Capt', 'Col', 'Countess', 'Don','Dona', 'Dr', 'Jonkheer', 'Lady','Major', 'Rev', 'Sir'], 'Other')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs':3, 'Other':4})

    # Age
    df.loc[df["Age"] < 1, "Age"] = df["Age"] * 100
    meanAge = df[['Title', 'Age']].groupby(['Title']).mean()
    for index, row in meanAge.iterrows():
        nullIndex = df[(df.Title == index) & (df.Age.isnull())].index
        df.loc[nullIndex, 'Age'] = row[0]

    df['AgeCategory'] = pd.qcut(df.Age, 8, labels=range(1, 9))
    df.AgeCategory = df.AgeCategory.astype(int)

    # Cabin
    df.Cabin.fillna('N', inplace=True)
    df["CabinCategory"] = df["Cabin"].str.slice(start=0, stop=1)
    df["CabinCategory"] = df['CabinCategory'].map({ "N": 0, "C": 1, "B": 2, "D": 3, "E": 4, "A": 5, "F": 6, "G": 7, "T": 8 })

    # Fare
    df.Fare.fillna(0, inplace=True)
    df['FareCategory'] = pd.qcut(df.Fare, 8, labels=range(1, 9))
    df.FareCategory = df.FareCategory.astype(int)


    # SibSp, Parch
    df['Family'] = df['SibSp'] + df['Parch'] + 1
    df.loc[df["Family"] > 4, "Family"] = 5

    df['IsAlone'] = 1
    df.loc[df['Family'] > 1, 'IsAlone'] = 0

    # Ticket
    df['TicketCategory'] = df.Ticket.str.split()
    df['TicketCategory'] = [i[-1][0] for i in df['TicketCategory']]
    df['TicketCategory'] = df['TicketCategory'].replace(['8', '9', 'L'], '8')
    df['TicketCategory'] = pd.factorize(df['TicketCategory'])[0] + 1

    df.drop(['PassengerId', 'Ticket', 'Cabin', 'Fare', 'Name', 'Age', 'SibSp', 'Parch'], axis=1, inplace=True)

    return df

train = feature_engineering(train)
test = feature_engineering(test)

train.info()
test.info()

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = train.drop('Survived', axis=1).values
target = train['Survived'].values

x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.4, stratify=target, random_state=0)

# Test 1
rf = RandomForestClassifier(n_estimators=50, criterion="entropy", max_depth=5, oob_score=True, random_state=10)
rf.fit(x_train, y_train)
prediction = rf.predict(x_valid)

length = y_valid.shape[0]
accuracy = accuracy_score(prediction, y_valid)
print(f'총 {length}명 중 {accuracy * 100:.3f}% 정확도로 생존을 맞춤')

# Test 2
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

RF_classifier = RandomForestClassifier()

RF_paramgrid = {
    'max_depth' : [6, 8, 10, 15],
    'n_estimators': [50, 100, 300, 500, 700, 800, 900],
    'max_features': ['sqrt'],
    'min_samples_split': [2, 7, 15, 30],
    'min_samples_leaf': [1, 15, 30, 60],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
    }


k_fold = StratifiedKFold(n_splits=5)
RF_classifiergrid = GridSearchCV(RF_classifier, param_grid = RF_paramgrid, cv=k_fold, scoring="accuracy", n_jobs= -1, verbose=1)

RF_classifiergrid.fit(x_train, y_train)

rf = RF_classifiergrid.best_estimator_

# Best Accuracy Score
print(RF_classifiergrid.best_score_)
print(RF_classifiergrid.best_params_)
print(RF_classifiergrid.best_estimator_)

submission = pd.read_csv(r'titanic_gender_submission.csv')
prediction = rf.predict(test)
submission['Survived'] = prediction
submission.to_csv(r'titanic_submission_20200414_last.csv', index=False)

# Feature Importance 확인. 생존률에 영향을 미친 정보 확인 
train.drop(['Survived'], axis=1, inplace=True)

import matplotlib.pyplot as plt
from pandas import Series

feature_importance = rf.feature_importances_
Series_feat_imp = Series(feature_importance, index=train.drop(['Survived'], axis=1).columns)

plt.figure(figsize=(8, 8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt.show()

총 357명 중 83.473% 정확도로 생존을 맞춤
Fitting 5 folds for each of 1792 candidates, totalling 8960 fits


[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 525 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done 875 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1325 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1875 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 2525 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 3275 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 4125 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 5075 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 6125 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 7275 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 8525 tasks      | elapsed: 11.1min
